# Find Student Acommodation in Melbourne (For International Students) - Mebourne Backpack
With several campuses scattered throughout the central business district, RMIT is surrounded by bustling shopping, community, public transport, and acommodation. Enrolled at RMIT and want to live close to campus?

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
from csv import writer

!pip install geopy
!pip install folium

import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster

# run a python file 
%run ../src/cleaning.py

# RMIT UNIVERSITY - MELBOURNE CITY CAMPUS - Geocoding Single Address
+ Phone number: +61 3 9925 2000
+ Website: https://www.rmit.edu.au
------------------------------------
### Address: 124 La Trobe St, Melbourne VIC 3000, Australia
+ Unilodge Carlton: 9 min walk
+ Unilodge Victoria: about 24 min public transport
+ Unilodge Lincoln: 9 min walk
+ Dwell: 5 min tram, 15 min walk

In [2]:
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode("124 La Trobe St, Melbourne VIC 3000, Australia")

print(location.address)
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

La Trobe Street, Melbourne Innovation District, Melbourne, City of Melbourne, Victoria, 3000, Australia
Latitude = -37.80874, Longitude = 144.966074


# [DWELL MELBOURNE STUDENT LIVING](https://www.dwellstudent.com.au/melbourne/#room)  (15 mins walk)

In [3]:
url= "https://www.dwellstudent.com.au/melbourne/#rooms"
page = requests.get(url)

scrape = soup(page.content, 'html.parser')
lists = scrape.find_all('div', class_="col-md-6 item")

with open('housing.csv', 'w', encoding='utf8', newline='') as f:
    thewriter = writer(f)
    header = ['title', 'icons', 'price', 'image']
    thewriter.writerow(header)

    for list in lists:
        title = list.find('div', class_="title")
        icons = list.find('div', class_="icon-number") #living, bed, bath numbers
        price = list.find('div', class_="price-wrapper")
        image = list.find('div', class_="img bg-cover-center")
        
        info = [title, icons, price, image]
        thewriter.writerow(info)
        
print(info)

[<div class="title">1 Bedroom Apartment</div>, <div class="icon-number">
<div class="icon living">1</div>
<div class="icon bed">1</div>
<div class="icon bath">1</div>
</div>, <div class="price-wrapper">
<div class="text">From</div>
<div class="price">
                    $529                  </div>
</div>, <div class="img bg-cover-center" style="background-image: url(https://www.dwellstudent.com.au/wp-content/uploads/2019/10/1-Br-Apartment-livingroom-h-768x512.jpg);">
<img class="img-mask" src="https://www.dwellstudent.com.au/wp-content/uploads/2019/10/1-Br-Apartment-livingroom-h-768x512.jpg"/>
</div>]


In [34]:
df = pd.read_csv('housing.csv')
df.head(3)

,title,icons,price,image
0,"<div class=""title"">5 Bedroom Apartment</div>","<div class=""icon-number"">\n<div class=""icon li...","<div class=""price-wrapper"">\n<div class=""text""...","<div class=""img bg-cover-center"" style=""backgr..."
1,"<div class=""title"">5 Bedroom Apartment Deluxe<...","<div class=""icon-number"">\n<div class=""icon li...","<div class=""price-wrapper"">\n<div class=""text""...","<div class=""img bg-cover-center"" style=""backgr..."
2,"<div class=""title"">4 Bedroom Apartment Deluxe<...","<div class=""icon-number"">\n<div class=""icon li...","<div class=""price-wrapper"">\n<div class=""text""...","<div class=""img bg-cover-center"" style=""backgr..."


In [35]:
import re
df['title'] = df['title'].str.replace(r'<[^<>]*>', '', regex=True)
df['icons'] = df['icons'].str.replace(r'<[^<>]*>', '', regex=True)
df['price'] = df['price'].str.replace(r'<[^<>]*>', '', regex=True).replace('\n', '', regex=True).replace(' ', '', regex=True)
df['image'] = df['image'].replace('<div class="img bg-cover-center" style="background-image: url\(', '', regex=True).replace('"/>\n</div>', '', regex=True).replace('\);">\n<img class="img-mask" src="', '', regex=True)

# added static columns inside the dataframe about Dwell Viallgae in Melbourne location
df['name'] = 'Dwell'
df['address'] = '5-17 Flemington Road, North Melbourne, VIC - 3051, Australia'
df['link'] = 'https://www.dwellstudent.com.au/melbourne/#rooms'
df['transportation'] = '5 min tram, 15 min walk'

# Extract Last n characters from right of the column in pandas:
df['price'] = df['price'].str[-3:]

# split icons column by space into 3 different cols and delete icons column from the dataframe
df = df.join(df.pop('icons').str.split(expand=True))
#rename specific column names
df = df.rename(columns = {0:'living', 1:'bed', 2:'bath'})


# [Unilodge Carlton](https://www.unilodge.com.au/student-accommodation-melbourne/carlton/) - 6 min walk

### Standard rooms include: 
+ a single or queen-sized bed, 
+ ensuite bathroom, 
+ unlimited wifi,
+ air conditioning. 
+ a shared kitchen the Mensa and an adjoined dining area, 
+ an outdoor courtyard with BBQ, 
+ a 24-hour gym and on-site staff.

### Travel times to the RMIT University:

+ From Student Living on Campus: 1-minute walk
+ From Student Living on Swanston: 4-minute walk
+ From Student Living - 570 Swanston St: 4-minute walk
+ From Student Living on A’Beckett: 6-minute walk
+ From Student Living on Lonsdale: 5-minute walk
+ From Student Living on Flinders: 11-minute walk
+ From Student Living on Cobden: 9-minute walk
+ From UniLodge Lincoln House: 6-minute walk
+ From Student Living D1: 6-minute walk

It’s easy to get around Melbourne thanks to the inclusive public transport system. With your choice of trains, trams and buses, you’ll be able to explore every inch of the city. Hop on Victoria’s public transport website to plan your journey ahead of time.

### LOCAL ATTRACTIONS
Continuously topping the Most Liveable City list, Melbourne is a city that knows how to put on a show. With historic buildings and beautiful gardens to explore, and culture exhibits and art galleries to see, there’s something here for everyone to enjoy. Walk through the Royal Botanic Gardens or check out the iconic Flinders Street Railway Station. The Queen Victoria Market is well worth a visit, spanning over 2 city blocks with food, clothing and more on offer.

Melbourne is renowned for its enviable dining scene. You’ll find incredible cafés, bars and restaurants in every suburb you visit. No matter where in the world you come from, you’re sure to find delicious eats that remind you of home.

### FACILITIES
Everything is easier when you stay at UniLodge. Our properties all come fully furnished with all you need to live comfortably. Complete with security features for your peace of mind, it’s the perfect place to base yourself while studying.

In [36]:

# uniLodge carlton - standard room
df.loc[13] = ['queen bed - standard', 269, 'https://image-tc.galaxy.tf/wijpeg-1ixp0jyz7yn92t0h4b9vq8t5r/wide.jpg?crop=5%2C0%2C711%2C400', 'uniLodge carlton', '609 Swanston Street, Carlton Victoria 3053 Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=382', '6-minute walk', 0, 1, 1]
df.loc[14] = ['single bed - standard', 269, 'https://image-tc.galaxy.tf/wijpeg-70pztniido0jw7r2b5rxomrq/wide.jpg?crop=5%2C0%2C711%2C400', 'uniLodge carlton', '609 Swanston Street, Carlton Victoria 3053 Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=382', '6-minute walk', 0, 1, 1]

# [STUDENT LIVING - 570 SWANSTON](https://www.unilodge.com.au/student-accommodation-melbourne/570-swanston)

In [37]:
# 1 Bedroom Apartment - Student Living - 570 Swanston
df.loc[15] = ['1 Bedroom Apartment - standard', 285, 'https://image-tc.galaxy.tf/wijpeg-br3qxv9j7zf3ckj9wpi5sopg7/wide.jpg?crop=0%2C53%2C1024%2C576', '570 Swanston', '570 Swanston St, Carlton VIC 3053, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=69', '4-minute walk', 0, 1, 1]

# 2 Bedroom Apartment - Student Living - 570 Swanston
df.loc[16] = ['2 Bedroom Apartment - standard', 320, 'https://image-tc.galaxy.tf/wijpeg-dqk5jil81r5vxcjehwn1emx1c/wide.jpg?crop=0%2C42%2C800%2C450', '570 Swanston', '570 Swanston St, Carlton VIC 3053, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=70', '4-minute walk', 0, 2, 1]
df.loc[17] = ['2 Bedroom Apartment - medium', 340, 'https://image-tc.galaxy.tf/wijpeg-dqk5jil81r5vxcjehwn1emx1c/wide.jpg?crop=0%2C42%2C800%2C450', '570 Swanston', '570 Swanston St, Carlton VIC 3053, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=70', '4-minute walk', 0, 2, 1]
df.loc[18] = ['2 Bedroom Apartment - large', 355, 'https://image-tc.galaxy.tf/wijpeg-dqk5jil81r5vxcjehwn1emx1c/wide.jpg?crop=0%2C42%2C800%2C450', '570 Swanston', '570 Swanston St, Carlton VIC 3053, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=70', '4-minute walk', 0, 2, 1]

# 1 Bedroom Apartment - Student Living - 570 Swanston
df.loc[16] = ['2 Bedroom Apartment - standard', 320, 'https://image-tc.galaxy.tf/wijpeg-dqk5jil81r5vxcjehwn1emx1c/wide.jpg?crop=0%2C42%2C800%2C450', '570 Swanston', '570 Swanston St, Carlton VIC 3053, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=70', '4-minute walk', 0, 2, 1]
df.loc[17] = ['2 Bedroom Apartment - medium', 340, 'https://image-tc.galaxy.tf/wijpeg-dqk5jil81r5vxcjehwn1emx1c/wide.jpg?crop=0%2C42%2C800%2C450', '570 Swanston', '570 Swanston St, Carlton VIC 3053, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=70', '4-minute walk', 0, 2, 1]
df.loc[18] = ['2 Bedroom Apartment - large', 355, 'https://image-tc.galaxy.tf/wijpeg-dqk5jil81r5vxcjehwn1emx1c/wide.jpg?crop=0%2C42%2C800%2C450', '570 Swanston', '570 Swanston St, Carlton VIC 3053, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=70', '4-minute walk', 0, 2, 1]

# 3 Bedroom Apartment - Student Living - 570 Swanston
df.loc[19] = ['3 Bedroom Apartment - standard', 445, 'https://image-tc.galaxy.tf/wijpeg-7b1iabbnxt4i0nb6hpkwuftqg/wide.jpg?crop=0%2C53%2C1024%2C576', '570 Swanston', '570 Swanston St, Carlton VIC 3053, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=71', '4-minute walk', 0, 3, 1]


# [UNILODGE VICTORIA UNIVERSITY](https://www.unilodge.com.au/student-accommodation-melbourne/vu) - about 24 min public transport

In [38]:
# [6 Bedroom Apartments - UniLodge Victoria University](https://www.reserve.unilodge.com.au/categoryInfo.html?category=68)
df.loc[20] = ['6 Bedroom Apartment - regular', 199, 'https://image-tc.galaxy.tf/wijpeg-4wrik6sohjn7xeze1wfcfsl2y/wide.jpg?crop=0%2C53%2C1024%2C576', 'UniLodge Victoria', '101 Ballarat Rd, Footscray VIC 3011, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=68', 'about 24 min public transport', 0, 6, 2]

# [Studio Apartments - UniLodge Victoria University](https://www.reserve.unilodge.com.au/categoryInfo.html?category=68)
df.loc[21] = ['Studio Apartments - regular', 'from 150', 'https://image-tc.galaxy.tf/wijpeg-7r3e5osm0utdmobnad97z6th9/wide.jpg?crop=0%2C0%2C1600%2C900', 'UniLodge Victoria', '101 Ballarat Rd, Footscray VIC 3011, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=66#room9436', 'about 24 min public transport', 0, 1, 1]

# [Studio Apartments - UniLodge Victoria University](https://www.reserve.unilodge.com.au/categoryInfo.html?category=68)
df.loc[22] = ['Studio Apartments - twin share', 'from 150', 'https://image-tc.galaxy.tf/wijpeg-4usn1iz0caxxaapw29ezv4d94/wide.jpg?crop=0%2C0%2C1842%2C1036', 'UniLodge Victoria', '101 Ballarat Rd, Footscray VIC 3011, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=66#room9436', 'about 24 min public transport', 0, '1 bunk bed', 1]

# [Studio Apartments - UniLodge Victoria University](https://www.reserve.unilodge.com.au/categoryInfo.html?category=68)
df.loc[22] = ['Studio Apartments - delux', 'from 150', 'https://image-tc.galaxy.tf/wijpeg-2ykwtr5lpvcnr1lgliaolwlg9/wide.jpg?crop=0%2C0%2C1600%2C900', 'UniLodge Victoria', '101 Ballarat Rd, Footscray VIC 3011, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=66#room9436', 'about 24 min public transport', 0, '1 double bed', 1]

# [Studio Apartments - UniLodge Victoria University](https://www.reserve.unilodge.com.au/categoryInfo.html?category=68)
df.loc[23] = ['Studio Apartments - accessible', 'from 150', 'https://image-tc.galaxy.tf/wijpeg-ebfm537cmd11ydagj75p3njrc/wide.jpg?crop=0%2C77%2C1600%2C900', 'UniLodge Victoria', '101 Ballarat Rd, Footscray VIC 3011, Australia', 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=66#room9436', 'about 24 min public transport', 0, '1 queen bed', 1]


# Converting addresses to geographic information — Latitude and Longitude

In [39]:
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent="scrape")

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
df['location'] = df['address'].apply(geocode)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
df.head()

,title,price,image,name,address,link,transportation,living,bed,bath,location,point
0,5 Bedroom Apartment,285,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"
1,5 Bedroom Apartment Deluxe,355,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"
2,4 Bedroom Apartment Deluxe,339,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",4,4,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"
3,3 Bedroom Apartment,309,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,3,1,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"
4,3 Bedroom Apartment Large,387,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,3,1,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)"


In [40]:
print(df.columns)
# split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)
df.head()

Index(['title', 'price', 'image', 'name', 'address', 'link', 'transportation',
       'living', 'bed', 'bath', 'location', 'point'],
      dtype='object')


,title,price,image,name,address,link,transportation,living,bed,bath,location,point,latitude,longitude,altitude
0,5 Bedroom Apartment,285,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0
1,5 Bedroom Apartment Deluxe,355,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,5,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0
2,4 Bedroom Apartment Deluxe,339,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",4,4,2,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0
3,3 Bedroom Apartment,309,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,3,1,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0
4,3 Bedroom Apartment Large,387,https://www.dwellstudent.com.au/wp-content/upl...,Dwell,"5-17 Flemington Road, North Melbourne, VIC - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,3,1,"(Flemington Road, North Melbourne, Melbourne, ...","(-37.7966706, 144.9511786, 0.0)",-37.796671,144.951179,0.0


In [41]:
# drop unecessary columns
print(f'List of existing columns: {df.columns.to_list()}\n\n')
df = df.drop(['location', 'point', 'altitude', 'living'], axis=1)
print(f'List of existing columns after dropping: {df.columns.to_list()}\n\n')

List of existing columns: ['title', 'price', 'image', 'name', 'address', 'link', 'transportation', 'living', 'bed', 'bath', 'location', 'point', 'latitude', 'longitude', 'altitude']


List of existing columns after dropping: ['title', 'price', 'image', 'name', 'address', 'link', 'transportation', 'bed', 'bath', 'latitude', 'longitude']




In [42]:
print(f'NUMBER OF CATEGORIES: {df.link.nunique()}; \n\nUNIQUE NAMES OF THE CATEGORIES {df.link.unique()}\n\n\n')

NUMBER OF CATEGORIES: 7; 

UNIQUE NAMES OF THE CATEGORIES ['https://www.dwellstudent.com.au/melbourne/#rooms'
 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=382'
 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=69'
 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=70'
 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=71'
 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=68'
 'https://www.reserve.unilodge.com.au/categoryInfo.html?category=66#room9436']





In [43]:
boolean_condition = df.image.str.contains('dwell', regex=False)
column_name = 'image'
new_value = 'https://www.dwellstudent.com.au/wp-content/uploads/2019/10/5-bed-living-room-600x353-deluxe.jpg'

df.loc[boolean_condition, column_name] = new_value

# Performe basic cleaning

In [44]:
"""
        Remove extra leading and tailing whitespace from the data.
        Lower case all contents inside the dataframe
"""

whitespace_remover(df)
# Cast all values inside the dataframe (except the columns' name) into lowercase
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

conditions = [(df["name"] == "dwell"), (df["name"] == "unilodge carlton"), (df["name"] == "570 swanston"), (df["name"] == "unilodge victoria")]


category_id = [1, 2, 3, 4]

description = ['BOOK NOW TO ENJOY OUR LIMITED TIME SPECIAL RATES!. SHORT STAY AT OUR LONG STAY RATES. FREE AIRPORT TRANSFER, LAUNDRY CREDIT AND SHOPPING VOUCHER. PARKING SPOT AT $250 PER MONTH. REFER A FRIEND AND GET $600. Your rent includes electricity, gas, water and unlimited wireless internet. dwell Scholarships are available for eligible students living in Australia. dwell VMC is an endorsed preferred provider for RMIT University and an approved provider for University of Melbourne. We also provide RMIT University scholarships to selected students! Your parent can now stay with us while you get settled in'
               , 'UniLodge Carlton is located right next to RMIT University in the heart of Melbourne, and conveniently located within walking distance of the city\'s iconic landmarks and tourist destinations such as the Queen Victoria Market and the Royal Exhibition Building. It is an ideal base for students to locate allowing easy access to the Lygon Street Italian precinct, restaurants at Gertrude Street and the Melbourne Central Shopping centre. Standard rooms include a single or queen-sized bed, ensuite bathroom, unlimited wifi and air conditioning. There are heaps of facilities at the property you can enjoy such as a shared kitchen the Mensa and an adjoined dining area, an outdoor courtyard with BBQ, a 24-hour gym and on-site staff.'
               , 'Perfectly located in the heart of all the action, Student Living - 570 Swanston offers accommodation in a selection of fully furnished apartments ranging from one, two and three bedrooms to students or city workers. All apartments offer open plan living, a functional kitchenette, bathroom, and a private balcony next to the living area with some apartments facing the Lincoln Square Park.It’s only a few minutes\' walk to RMIT University, public transport, parks, cafes, restaurants, and a short tram ride into Melbourne city. If you do need to get around Melbourne, Student Living - 570 Swanston has the convenience of a tram stop right at the front door! There is also on-site car parking (at an additional cost) should you need to utilise one. Close by you have Lygon Street precinct – home to Melbourne’s Italian quarter with an abundance of restaurants and eateries. 10 minutes by foot has you in either the main city centre or in China Town – take your pick for shopping or eating.'
               , 'BEST VALUE IN TOWN! FROM $150 PW INCLUDING WIFI AND UTILITIES. Looking for places to stay without paying CBD prices? You can live and study in one of Melbourne’s most exciting inner-city suburbs! Victoria University is a world-class institute located in Footscray, just 5 km on the western fringe of the city of Melbourne. (fully furnished including utilities and unlimited wifi!). A range of room types from twin, 2 and 6 bed apartments, to self-contained studios. No lease – no fuss! On site staff providing excellent safety and security 24/7. Study rooms, games rooms, cinema room, entertainment galore. Rooftop terrace, basketball half-court and BBQ facilitie. Our very own onsite café barista! Weekly events and activities to help you make new friends']

rating = [4.347826, 2.485714, 4.857143, 3.676471]

df['category_id'] = np.select(conditions, category_id, default='black')
df['description'] = np.select(conditions, description, default='black')
df['rating'] = np.select(conditions, rating, default='black')



whitespace_remover(df)
# Cast all values inside the dataframe (except the columns' name) into lowercase
df = df.applymap(lambda s: s.lower() if type(s) == str else s)

df.to_csv('../data/interim/housing.csv', index=False)
df

,title,price,image,name,address,link,transportation,bed,bath,latitude,longitude,category_id,description,rating
0,5 bedroom apartment,285,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,2,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
1,5 bedroom apartment deluxe,355,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",5,2,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
2,4 bedroom apartment deluxe,339,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",4,2,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
3,3 bedroom apartment,309,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,1,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
4,3 bedroom apartment large,387,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",3,1,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
5,2 bedroom studio,298,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",2,1,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
6,2 bedroom studio with balcony,229,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",2,1,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
7,2 bedroom apartment,329,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",2,1,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
8,1 bedroom suite,439,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",1,1,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826
9,1 bedroom studio deluxe,439,https://www.dwellstudent.com.au/wp-content/upl...,dwell,"5-17 flemington road, north melbourne, vic - 3...",https://www.dwellstudent.com.au/melbourne/#rooms,"5 min tram, 15 min walk",1,1,-37.796671,144.951179,1,book now to enjoy our limited time special rat...,4.347826


In [45]:
df.description.nunique()

4

In [46]:
df.to_json('../data/interim/housing.json', orient='records')
print("successfully saved")

successfully saved


In [47]:
reviews = pd.read_csv('../data/interim/reviews/reviews.csv')
reviews = reviews.drop(columns=['Unnamed: 4', 'Unnamed: 5'])
reviews.to_csv('../data/interim/reviews.csv', index=False)
print("successfully saved")

successfully saved


In [48]:
reviews

,comment,rating,link,category_id
0,"Friendly staff, convenient location near Victo...",5,https://www.facebook.com/unilodge.vu/reviews,4
1,I stay here for 2 year and it make me feel hom...,5,https://www.facebook.com/unilodge.vu/reviews,4
2,I makes a lot of friend here too and having a ...,5,https://www.facebook.com/unilodge.vu/reviews,4
3,The staff is very polite and gentle specially ...,5,https://www.facebook.com/unilodge.vu/reviews,4
4,5 Star service experience here at UniLodge @ V...,5,https://www.facebook.com/unilodge.vu/reviews,4
5,The management is really terrible and there is...,1,https://www.facebook.com/unilodge.vu/reviews,4
6,The management at UniLodge is appalling to say...,1,https://www.facebook.com/unilodge.vu/reviews,4
7,The room and the housemates are really terribl...,1,https://www.facebook.com/unilodge.vu/reviews,4
8,Highly recommend this place to anyone. So clos...,5,https://www.facebook.com/unilodge.vu/reviews,4
9,The staff are extremely nice and very helpful ...,5,https://www.facebook.com/unilodge.vu/reviews,4
